In [2]:
%run lit_utility_functions.ipynb
%un set_up.py

#Info: tags from WoS https://images.webofknowledge.com/images/help/WOS/hs_wos_fieldtags.html

#Subdirectory where tab-delimited files of wos outputs were saved
scoping_4_datdir = Path(scopdir, 'scoping_4_wos')

#Pickle were loaded data will be saved
scoping_4_docset_pickle = Path(scoping_4_datdir, 'scoping_4_docset.pickle') 

#csv table where references' title, journal, year and DOI are written
reflist_csv = Path(scoping_4_datdir, 'scoping_4_shortmetadata.csv')

#csv table with 200 sampled articles
reflist_csv_sample = Path(scoping_4_datdir, 'scoping_4_shortmetadata_sample.csv')

<>:57: SyntaxWarning: invalid escape sequence '\d'
C:\Users\messa\AppData\Local\Temp\ipykernel_16180\894219484.py:57: SyntaxWarning: invalid escape sequence '\d'
  reflist_dict[i] = [re.sub("[^a-zA-Z\d\s]", "", ref.title.replace('\n', ' ').lower()),
C:\Users\messa\AppData\Local\Temp\ipykernel_16180\894219484.py:57: SyntaxWarning: invalid escape sequence '\d'
  reflist_dict[i] = [re.sub("[^a-zA-Z\d\s]", "", ref.title.replace('\n', ' ').lower()),


ModuleNotFoundError: No module named 'litstudy'

ModuleNotFoundError: No module named 'litstudy'

In [2]:
#~~~~~~~~~~~~~ Read and compile references from the WoS search ~~~~~~~~~~~~~~~~~~
#into a single document set (lit_study format)
reflist = rpickle_bibdocset(in_dirpath=scoping_4_datdir, 
                  in_pattern=".*savedrecs(\([0-9]{1,2}\))*[.]bib", 
                  out_pickle=scoping_4_docset_pickle)

In [3]:
#~~~~~~~~~~~~~ Get titles and DOIs from test list ~~~~~~~~~~~~~~~
testlist_title_dois = get_testlist(library_id='4842799', api_key_path=Path(srcdir, 'zotero_key.txt'))

In [8]:
#~~~~~~~~~~~~~~ Write basic metadata from reference list to csv ~~~~~~~~~~~~~~~~~~
#Get all dois and titles in references returned from search
reflist_pd = tabulate_searchlist(in_reflist=reflist, out_csvpath=reflist_csv) 

print(len(set(reflist_pd.title)))

if not reflist_csv_sample.exists():
    reflist_pd_sample = reflist_pd.sample(100)
    reflist_pd_sample.to_csv(reflist_csv_sample)
else: 
    reflist_pd_sample = pd.read_csv(reflist_csv_sample)

78279


In [5]:
#~~~~~~~~~~~~~~ Check which items in test list were retrieved through the search ~~~~~~~
returned = {}
not_returned = {}

for k, v in testlist_title_dois.items():
    if (re.sub("[^a-zA-Z\d\s]", "", v[0].lower()) in set(reflist_pd.title)) or (v[1] in set(reflist_pd.doi)):
        returned[k] = v[0]
    else:
        not_returned[k] = v[0]

In [ ]:
#Compute the number of included and excluded references depending on all catgeories for each document
#Format as a table to be examined
wos_initlist_catsample_format = pd.read_csv(catsamplecsv_path)

wos_catsample_stats = pd.melt(frame=wos_initlist_catsample_format,
        id_vars=['Unnamed: 0', 'Inclusion'],
        value_vars = wos_cat_split.columns,
        #var_name='wos_catnum',
        value_name = 'wos_catnum'
        )

wos_catsample_stats['wos_catnum'] = wos_catsample_stats['wos_catnum'].replace(r"^ +| +$", r"", regex=True)

wos_catsample_stats = wos_catsample_stats.groupby(['wos_catnum', 'Inclusion']).size().to_frame('N').reset_index().\
    pivot(index='wos_catnum', columns='Inclusion')

wos_catsample_stats.columns = wos_catsample_stats.columns.droplevel()
wos_catsample_stats.reset_index(inplace=True)
wos_catsample_stats = wos_catsample_stats.rename(columns={0:"excluded", 1:"included"}).\
    fillna(0)
wos_catsample_stats['N'] = wos_catsample_stats['excluded'] + wos_catsample_stats['included']
wos_catsample_stats.sort_values(['included','N'], ascending=[False, False]).\
to_csv(Path(scopdir, 'wos_search1_catsample_stats.csv'))

In [6]:
print(len(not_returned))
not_returned

19


{'9WNWDU83': 'Generalized models of riverine fish hydraulic habitat',
 'FM669F58': 'Estimating Minimum Instream Flow Requirements for Minnesota Streams from Hydrologic Data and Watershed Characteristics',
 'QSA6R86M': 'The Applicability of Instream Flow Incremental Methodology for Impact Assessment in Newfoundland',
 '8LLPFL3Q': 'Some Environmental Aspects of the Management of Water Supplies for the Region of Madrid',
 'BD4MS6ET': 'Application of the Instream Flow Incremental Methodology to conservation flow for freshwater fishes in Japan',
 '5ABFNPLW': 'Effects of experimental ramping rate of invertebrate community of a regulated river',
 'FBNIZFMX': 'Flow-recruitment relationships for Shoal Chub (Macrhybopsis hyostoma) and implications for managing environmental flows',
 'XXWDW65Y': 'The basic flow: an alternative approach to calculate minimum environmental instream flows',
 'KF78K6WX': 'Comparison of discharge methods and habitat optimization for recommending instream flows to prote

In [ ]:
#Generalized models of riverine fish hydraulic habitat - not returned
#Estimating Minimum Instream Flow Requirements for Minnesota Streams from Hydrologic Data and Watershed Characteristics - not returned in WoS
#The Applicability of Instream Flow Incremental Methodology for Impact Assessment in Newfoundland not returned in WoS
#Some Environmental Aspects of the Management of Water Supplies for the Region of Madrid - not returned in WoS
#Application of the Instream Flow Incremental Methodology to conservation flow for freshwater fishes in Japan - Not returned - Conference Paper
#Effects of instream enhancement structures on brown trout, Salmo trutta L., habitat availability in a channelized boreal river: a PHABSIM approach - not returned in WoS
#Flow-recruitment relationships for Shoal Chub (Macrhybopsis hyostoma) and implications for managing environmental flows - not returned in WoS (maybe because of date)
#Effects of experimental ramping rate of invertebrate community of a regulated river - not returned conference paper
#Comparison of discharge methods and habitat optimization for recommending instream flows to protect fish habitat - not returned in WoS
#Relative bias of several fisheries instream flow methods - not returned
#Enhancement of benthic macroinvertebrates by minimum flow from a hydroelectric dam - not returned
#Integrated Procedure for Environmental Flow Assessment in Rivers - not returned in WoS
#Environmental Flow Assessment Using Water-Sediment Approach at the Sekampung River, Indonesia - not returned in WoS
#Predicting Salmonid Habitat–Flow Relationships for Streams from Western North America = not returned
#Enhancement of Fish Feeding and Growth after an Increase in Minimum Flow below the Conowingo Dam - not returned in WoS
#The basic flow: an alternative approach to calculate minimum environmental instream flows - not returned conference paper


#Hydrologic manipulations of the channelized Kissimmee river - Implications for restoration - RETURNED in WoS. Change string
#History, rationale, and lessons learned: Thresholds of potential concern in Kruger National Park river adaptive management - RETURNED in WoS. Not possible to capture.